## Denoising Diffusion Probalistic Model Implementation

## Importing Libraries and Modules

In [1]:
import os
from tqdm import tqdm

# Data processing imports
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Pytorch imports
import torch
import torchvision
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# Importing the models
from Unet import UNet
from Diffusion_model import Diffusion

## Data Preparation and Loading

In [3]:

## Load the multiple npy files and appending it to a list

directory = '../../00_data/Samples'
numpy_list = []

for file_name in os.listdir(directory):
    if file_name.endswith('.npy'):
        file_path = os.path.join(directory, file_name)
        image_np = np.load(file_path)

        numpy_list.append(image_np)



In [4]:


def save_images(generated, path, **kwargs):
    # Convert to numpy arrays
    generated = generated.to('cpu').numpy()

    # Determine the number of images
    num_images = generated.shape[0]
    # Calculate the number of rows needed (assuming 4 images per row for display)
    num_rows = (num_images + 3) // 4  # Round up division

    # Create a figure and axes
    fig, axes = plt.subplots(nrows=num_rows, ncols=4, figsize=(20, num_rows * 5))
    axes = axes.flatten()  # Flatten the axes array for easier indexing15

    # Adjust space between images
    plt.subplots_adjust(wspace=0.3, hspace=0.3)

    for i, gen_img in enumerate(generated):
        # Display generated images
        axes[i].imshow(gen_img.squeeze(), cmap='gray', **kwargs)
        axes[i].axis('off')
        axes[i].set_title(f"Generated {i+1}", fontsize=10)

    # In case the last row is not fully populated
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    # Save the figure
    fig.savefig(path, bbox_inches='tight')
    plt.close(fig)


In [ ]:
# def save_images_individually(generated, directory):
#     # Convert to numpy arrays
#     generated = generated.detach().cpu().numpy()

#     # Determine the number of images
#     num_images = generated.shape[0]
    
#     # Ensure the directory exists
#     if not os.path.exists(directory):
#         os.makedirs(directory)

#     # Save each image individually
#     for i, gen_img in enumerate(generated):
#         # Remove the channel dimension if it's 1 (for grayscale images)
#         if gen_img.shape[0] == 1:
#             gen_img = gen_img.squeeze(0)
#         # Convert to 0-255 range and uint8 type
#         gen_img = ((gen_img - gen_img.min()) * (1/(gen_img.max() - gen_img.min()) * 255)).astype('uint8')
#         img = Image.fromarray(gen_img)
#         img.save(os.path.join(directory, f"generated_{i+1}.png"))

## Downsample the images to 64x64 and normalize the images
* Downsample the images to 128x128 will be better but due to computational constraints, I am downsampling to 64x64

In [6]:
# Convert the list of numpy arrays to a numpy array
x_train = np.array(numpy_list)

# Convert to a list of PIL images, resize each, and then convert back to a NumPy array
x_train = np.array([np.array(Image.fromarray(img.squeeze()).resize((64, 64))) for img in x_train])

# The new shape will be (10000, 64, 64). If you need to add the channel dimension back:
x_train = x_train[:, np.newaxis, :, :]
x_train = torch.Tensor(x_train)

print(x_train.shape)



torch.Size([10000, 1, 64, 64])


In [7]:
type(x_train)

torch.Tensor

## Making Data Loaders for the training data

In [8]:
# Create a DataLoader
dataset = TensorDataset(x_train)
dataloader = DataLoader(dataset, batch_size=10)


In [19]:
# Set the device
device = "cuda"

# Initialize the model and move it to the device
model = UNet().to(device)

# Initialize the optimizer
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

# Initialize the loss function
mse = nn.MSELoss()

# Training loop
diffusion = Diffusion(img_size=64, device=device)
l = len(dataloader)

In [20]:
# Set the model to training mode, I already did this for 12 epochs so I will start from 13
start_epoch = 13
epochs = 100

#if model exists, load it
if os.path.exists('./Weights/Diff_ckpt_1.pt'):
    model.load_state_dict(torch.load('./Weights/Diff_ckpt_1.pt'))
    print("Model loaded")


Model loaded


## Training the Model

In [21]:
# Loop through each epoch starting from start_epoch up to epochs
for epoch in range(start_epoch, epochs):
    # Print the current epoch number
    print(f"Starting epoch {epoch}:")
    # Initialize a progress bar for the data loader
    pbar = tqdm(dataloader)
    # Initialize average Mean Squared Error (MSE) for the epoch
    avg_mse = 0
    # Loop through each batch of images in the dataset
    for i, (images,) in enumerate(pbar): # Removed conditions for now to make the model a Denoising Diffusion Probabilistic Model (DDPM)
        # Move images to the designated computing device (e.g., GPU)
        images = images.to(device)
        # Sample timesteps for the diffusion process
        t = diffusion.sample_timesteps(images.shape[0]).to(device)
        # Corrupt images with noise and get the original noise
        x_t, noise = diffusion.noise_images(images, t)
        # Predict the noise using the model. Conditions are not used in this DDPM version.
        predicted_noise = model(x_t, t)
        # Calculate the mean squared error loss between the actual and predicted noise
        loss = mse(noise, predicted_noise)

        # Zero the gradients before running the backward pass
        optimizer.zero_grad()
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # Perform a single optimization step (parameter update)
        optimizer.step()

        # Update the progress bar with the current MSE
        pbar.set_postfix(MSE=loss.item())
        
        # Accumulate the MSE for averaging later
        avg_mse += loss.item()

    # After all batches are processed, print the average MSE for the epoch
    print(f'Average MSE: {avg_mse/1000:.5f}\n')
    # Sample images from the model to visualize progress
    sampled_images = diffusion.sample(model, n=images.shape[0])
    # Save the sampled images for this epoch
    save_images(sampled_images, os.path.join("Results", f"{epoch}.png"))
    # Save the model's state for potential later use or resuming training
    torch.save(model.state_dict(), os.path.join("Weights", f"Diff_ckpt_1.pt"))


Starting epoch 13:


100%|██████████| 1000/1000 [02:03<00:00,  8.11it/s, MSE=0.00151]


Average MSE: 0.00186

Sampling 10 new images....


999it [00:44, 22.34it/s]


Starting epoch 14:


...

999it [00:38, 26.16it/s]


Starting epoch 99:


100%|██████████| 1000/1000 [01:54<00:00,  8.76it/s, MSE=0.000923]


Average MSE: 0.00113

Sampling 10 new images....


999it [00:38, 26.25it/s]


In [ ]:
# Generate images using the trained model

# from tqdm.auto import tqdm
# import os

# # Load model weights
# if os.path.exists('./Weights/Diff_ckpt_1.pt'):
#     model.load_state_dict(torch.load('./Weights/Diff_ckpt_1.pt'))
#     print("Model loaded")
# model.to(device)

# # Set the model to evaluation mode
# model.eval()

# # Generate and save images
# with torch.no_grad():
#     for i in tqdm(range(100)):
#         sampled_images = diffusion.sample(model, n=10)
#         # save_images function needs to be defined, as well as a method to name each batch of generated images uniquely
#         save_images_individually(sampled_images, os.path.join("Results_individual", f"generated_{i}.png"))